# Core Imports

In [3]:
# Custom Imports
from polysaccharide import general
from polysaccharide.general import optional_in_place
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdcompare, rdconvert, rdkdraw, rdcompare, rdprops, rdbond, rdlabels

from polysaccharide.polymer import monomer as monoutils
from polysaccharide.polymer.monomer import MonomerInfo
from polysaccharide.polymer.management import PolymerManager

from polysaccharide.polymer import building
import mbuild as mb

# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

# File and chemistry type definitions

In [4]:
pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

lammps_path = Path('LAMMPS')
lammps_path.mkdir(exist_ok=True)

omm_path = Path('OpenMM')
omm_path.mkdir(exist_ok=True)

In [5]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Collating urethanes into collections and generating Interchange files

## Generating LAMMPS files via Interchange for both collections

In [4]:
from polysaccharide import OPENFF_DIR
from polysaccharide.charging.application import MolCharger

# specify forcefield
ff_name = 'openff-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# specify charging method
chg_method = 'Espaloma_AM1BCC'

In [ ]:
from tqdm.notebook import tqdm

success_ics = defaultdict(defaultdict)
failed_ics  = defaultdict(lambda : defaultdict(list))

for chemistry in chemistries:
    chem_path = coll_path / chemistry
    lmp_dir = lammps_path / chemistry
    lmp_dir.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    for mol_name, polymer in ( progress := tqdm(mgr.polymers.items()) ):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        try:
            chgr = MolCharger.subclass_registry[chg_method]()
            polymer.assert_charges_for(chgr, strict=True, return_cmol=False)

            sdf_path = polymer.structure_files_chgd[chg_method]
            cmol = polymer.charged_offmol_from_sdf(chg_method)
            offtop = Topology.from_molecules(cmol) # load topology from SDF file

            ic = forcefield.create_interchange(offtop, charge_from_molecules=[cmol])
            success_ics[chemistry][mol_name] = ic
            # ic.to_lammps(lmp_dir / f'{mol_name}.lammps')

        except AttributeError as a:
            print(a)

        except Exception as e:
            print(e)
            failed_ics[chemistry][e.__class__.__name__].append(mol_name)

In [ ]:
success_ics, failed_ics

# Running OpenMM simulations

## Defining utility functions

In [6]:
import openmm

from openmm import XmlSerializer
from openmm import System, Context, State
from openmm import Integrator, Force
from openmm.app import Simulation

DEFAULT_STATE_PARAMS : dict[str, bool] = {
    'getPositions'  : True,
    'getVelocities' : True,
    'getForces'     : True,
    'getEnergy'     : True,
    'getParameters' : True,
    'getParameterDerivatives' : False,
    'getIntegratorParameters' : False
}



def serialize_state_and_sys(sim : Simulation, out_dir : Path, out_name : str, state_params : dict[str, bool]=DEFAULT_STATE_PARAMS) -> None:
    '''For saving State and System info of a Simulation to disc'''
    sim_dict = {
        'system' : sim.system,
        'state' : sim.context.getState(**state_params)
    }
    
    for affix, save_data in sim_dict.items():
        save_path = out_dir / f'{out_name}_{affix}.xml'
        save_path.touch()

        with save_path.open('w') as file:
            file.write( XmlSerializer.serialize(save_data) )

def apply_state_to_context(state : State, context : Context) -> None:
    '''For applying saved State data to an existing OpenMM Simulation'''
    context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
    context.setPositions(state.getPositions())
    context.setVelocities(state.getVelocities())
    context.setTime(state.getTime())

    context.reinitialize(preserveState=True)    

def load_openmm_system(sys_path : Path, extra_forces : Optional[Union[Force, Iterable[Force]]]=None, sep_force_grps : bool=True, remove_constrs : bool=False) -> System:
    '''Load and configure a serialized OpenMM system, with optional additional parameters'''
    assert(sys_path.suffix == '.xml')
    with sys_path.open('r') as file:
        ommsys = XmlSerializer.deserialize(file.read())

    if extra_forces: # deliberately sparse to handle both Nonetype and empty list
        for force in extra_forces: 
            ommsys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(ommsys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(ommsys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            ommsys.removeConstraint(i)

    return ommsys

## Serialize OpenMM simulations to file

In [43]:
# specifying simulation and ensemble parameters
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole
from shutil import copyfile
from copy import deepcopy

from polysaccharide import filetree
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory


omm_dir = Path('OpenMM_no_sim')
omm_dir.mkdir(exist_ok=True)

# selecting simulation parameters and ensemble
sp_path = Path('debug_sim_NVT.json')
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

In [ ]:
# looping over all urethanes
omm_sims = defaultdict(defaultdict)
for chemistry, ic_dict in success_ics.items():
    mgr = PolymerManager(coll_path / chemistry)
    chem_dir = omm_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, interchange in ic_dict.items():
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        # creating simulation and associated files
        polymer = mgr.polymers[mol_name]
        sdf_path = polymer.structure_files_chgd[chg_method]

        # create and register simulation
        sim = ens_fac.create_simulation(interchange, sim_params)
        omm_sims[chemistry][mol_name] = sim

        # serialize Topology, System, and State for reloading
        sdf_out_path = mol_dir / f'{mol_name}_topology.sdf'
        copyfile(sdf_path, sdf_out_path)
        serialize_state_and_sys(sim, out_dir=mol_dir, out_name=mol_name)

## Loading simulations from file

In [7]:
from openmm import XmlSerializer
from openmm import Force

from polysaccharide import filetree
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory


# parameters
sep_force_grps : bool = True
remove_constrs : bool = True

chemistry = 'urethane'
mol_name = 'urethane_2'

# defining paths
sp_path = Path('debug_sim_NVT.json')
omm_dir = Path('OpenMM_no_sim')
omm_dir.mkdir(exist_ok=True)

ser_dir = omm_dir / chemistry / mol_name

# define files
state_file = ser_dir / f'{mol_name}_state.xml'
sys_file   = ser_dir / f'{mol_name}_system.xml'
top_file   = ser_dir / f'{mol_name}_topology.sdf'

# load Topology via OpenFF
offmol = Molecule.from_file(top_file)
offtop = Topology.from_molecules(offmol)
ommtop = offtop.to_openmm()

# define ensemble-specific forces and Integrator
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

integrator = ens_fac.integrator(sim_params)
forces     = ens_fac.forces   (sim_params)

# load and configure System
ommsys = load_openmm_system(sys_file, extra_forces=forces, sep_force_grps=sep_force_grps, remove_constrs=remove_constrs)

# putting it all together into a Simulation
sim = Simulation(
    topology=ommtop,
    system=ommsys,
    integrator=integrator,
    state=state_file
)

# load and apply State
# with state_file.open('r') as file:
#     ommstate = XmlSerializer.deserialize(file.read())
# apply_state_to_context(ommstate, sim.context)

In [51]:
eval_state = sim.context.getState(getEnergy=True)

In [53]:
eval_state.getPotentialEnergy().in_units_of(kilocalorie_per_mole)

Quantity(value=2270780.5927342256, unit=kilocalorie/mole)

In [ ]:
from openff.interchange import Interchange
from openmm import Integrator, Force

from openmm.unit import nanometer
from openff.units import unit as offunit

def create_simulation2(interchange : Interchange, integrator : Integrator, forces : Optional[Iterable[Force]]=None,
                        sep_force_grps : bool=True, remove_constrs : bool=True, combine_nonbonded_forces : bool=True) -> Simulation:
    '''Specifies configuration for an OpenMM Simulation - Interchange load alows many routes for creation'''
    openmm_sys = interchange.to_openmm(combine_nonbonded_forces=combine_nonbonded_forces) 
    openmm_top = interchange.topology.to_openmm()
    openmm_pos = interchange.positions.m_as(offunit.nanometer) * nanometer

    if forces: # deliberately sparse to handle both Nonetype and empty list
        for force in forces: 
            openmm_sys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(openmm_sys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(openmm_sys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            openmm_sys.removeConstraint(i)

    simulation = Simulation(openmm_top, openmm_sys, integrator)
    simulation.context.setPositions(openmm_pos)

    return simulation

## Probing NonbondedForce

In [58]:
nonbond = sim.system.getForce(0)

In [62]:
nonbond.getNumParticles()

828

In [73]:
nonbond.getParticleParameters(4)

[Quantity(value=0.02025200054049492, unit=elementary charge),
 Quantity(value=0.337953176162662, unit=nanometer),
 Quantity(value=0.45538911611061844, unit=kilojoule/mole)]

In [97]:
SIG = general.GREEK_LOWER['sigma']
EPS = general.GREEK_LOWER['epsilon']

records = []
for atom in sim.topology.atoms():
    charge, sigma, epsilon = nonbond.getParticleParameters(atom.index)
    records.append({
        'Atom name' : atom.name,
        'Element' : atom.element.symbol,
        'Partial charge' : charge,
        f'LJ_{SIG}' : sigma,
        f'LJ_{EPS}' : epsilon
    })

lj_dframe = pd.DataFrame.from_records(records)
lj_dframe

,Atom name,Element,Partial charge,LJ_σ,LJ_ε
0,C1x,C,0.7763530015945435 e,0.34806468869450646 nm,0.3635030558377792 kJ/mol
1,O1x,O,-0.6556079983711243 e,0.3039812205065809 nm,0.8795023257036865 kJ/mol
2,N1x,N,-0.47651898860931396 e,0.3206876023663901 nm,0.7016212989374017 kJ/mol
3,H1x,H,0.3168550133705139 e,0.11034276772973167 nm,0.058955968900150965 kJ/mol
4,C2x,C,0.02025200054049492 e,0.337953176162662 nm,0.45538911611061844 kJ/mol
...,...,...,...,...,...
823,H503x,H,0.020533999428153038 e,0.2583225710839196 nm,0.068656285380106 kJ/mol
824,H504x,H,0.020533999428153038 e,0.2583225710839196 nm,0.068656285380106 kJ/mol
825,N18x,N,-0.6973109841346741 e,0.3206876023663901 nm,0.7016212989374017 kJ/mol
826,C270x,C,0.5900880098342896 e,0.33996695084235345 nm,0.87864 kJ/mol


In [96]:
general.GREEK_LOWER['sigma']

'σ'

In [94]:
atoms[0].name

'C1x'

In [92]:
targ_elem = 'C'

targ_atoms = [
    (atom.index, atom)
        for atom in sim.topology.atoms()
            if atom.element.symbol == targ_elem
]

In [ ]:
for (index, atom) in targ_atoms:
    print(atom, nonbond.getParticleParameters(index))

In [8]:
sim.system.getForces()

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f49310a1050> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f49310a0b70> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f49310a1020> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f49310a3ab0> >]

In [11]:
bonded = sim.system.getForce(3)

## Evaluating starting structure energies

In [ ]:
NULL_ENERGY = 0.0*kilojoule_per_mole
PRECISION : int = 4

remove_constrs : bool = True

data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    for mol_name, sim in mol_dict.items():
        # assign and initialize unique force groups for simulation
        for i, force in enumerate(sim.system.getForces()):
            force.setForceGroup(i)
            # print(force.getName(), force.getForceGroup())
        sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

        # remove constraints
        if remove_constrs:
            for i in range(sim.system.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
                sim.system.removeConstraint(i)
            sim.context.reinitialize(preserveState=True)

        # extract total and component energies from OpenMM force groups
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }
        omm_energies = {}

        overall_state = sim.context.getState(getEnergy=True) # get total potential energy
        PE = overall_state.getPotentialEnergy()
        omm_energies['Total'] = PE

        KE = overall_state.getKineticEnergy()
        assert(KE == NULL_ENERGY)

        for i, force in enumerate(sim.system.getForces()):
            state = sim.context.getState(getEnergy=True, groups={i})
            force_name = force.getName().removesuffix('Force')
            omm_energies[force_name] = state.getPotentialEnergy()

        # reformat
        omm_energies_kcal = {}
        for contrib, energy_kj in omm_energies.items():
            energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
            omm_energies_kcal[f'{contrib} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)

        # compiling data
        data_dict = {**data_dict, **omm_energies_kcal}
        data_dicts.append(data_dict)

df = pd.DataFrame.from_records(data_dicts)
df.sort_values('Molecule', inplace=True, drop=True)
df.to_csv(omm_dir / 'Potential_energies_unconstrained.csv', index=False)

## Testing effect of removing constraints on energies

In [ ]:
ic = ic_dict['urethane_44']
integrator = ens_fac.integrator(sim_params)
forces = ens_fac.forces(sim_params)

In [ ]:
simm = create_simulation2(ic, integrator, forces=forces, sep_force_grps=True, remove_constrs=True, combine_nonbonded_forces=True)

In [ ]:
simm.system.getNumConstraints()

In [ ]:
overall_state = simm.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(simm.system.getForces()):
    state = simm.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    omm_energies[force_name] = state.getPotentialEnergy()

# reformat
omm_energies_kcal = {}
for contrib, energy_kj in omm_energies.items():
    energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
    omm_energies_kcal[f'{contrib} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)


In [ ]:
omm_energies_kcal

In [ ]:
for i in range(sim.system.getNumConstraints())[::-1]:
    print(sim.system.getConstraintParameters(i))
    sim.system.removeConstraint(i)

In [ ]:
list(range(5)[::-1])

In [ ]:
sim.system.getNumConstraints()

In [ ]:
sim.system.getConstraintParameters(359)

## Comparing ParmEd energy decomposition to native OpenMM force-group-based decomposition

In [ ]:
import parmed
from openmm.openmm import Force

NULL_ENERGY = 0.0*kilojoule_per_mole

sim = omm_sims['urethane']['urethane_41']
# assign and initialize unique force groups for simulation
for i, force in enumerate(sim.system.getForces()):
    force.setForceGroup(i)
    # print(force.getName(), force.getForceGroup())
sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

# energies from OpenMM force groups
print('\nOpenMM:')
print('='*30)
omm_energies = {}

## extract total energies for state
overall_state = sim.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total Potential Energy'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(sim.system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    pe = state.getPotentialEnergy()

    omm_energies[force_name] = pe
    print(f'{force_name} : {pe}')

## converting name to match with ParmEd for comparison
namemap = {
    'Nonbonded' : 'bond',
    'PeriodicTorsion' : 'angle',
    'HarmonicAngle' : 'dihedral',
    'HarmonicBond' : 'urey_bradley',
    'Total Potential Energy' : 'total'
}
compat_omm_energies = {
    namemap[contrib] : energy
        for contrib, energy in omm_energies.items()
}

total = sum(omm_energies.values(), start=NULL_ENERGY) # need "seed" to have Quantity datatype to sum
print(f'{general.GREEK_UPPER["delta"]}E_contrib: ', PE - total)

# ParmEd energy decomposition
print('\nParmEd:')
print('='*30)
parm_energies = {}
parm_struct = parmed.openmm.load_topology(sim.topology, sim.system)
for contrib, energy_val in parmed.openmm.energy_decomposition(parm_struct, sim.context).items():
    parm_energies[contrib] = energy = energy_val*kilocalorie_per_mole # assign proper units
    print(contrib, energy.in_units_of(kilojoule_per_mole))

## Minimizing and running single integration step, then evaluating energies from reporter

In [ ]:
import re
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

from polysaccharide.simulation import preparation
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory

STRIP_BEFORE_PARENS = re.compile(r'(.*?)(?=\s*\(.*\))')
PRECISION = 3 # number of decimals to round reported energies to

omm_dir = Path('OpenMM_no_sim')
omm_dir.mkdir(exist_ok=True)


# selecting simulation parameters and ensemble
sp_path = Path('debug_sim_NVT.json')
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

data_by_mol = []
omm_sims = defaultdict(defaultdict)
# looping over all urethanes
for chemistry, ic_dict in success_ics.items():
    chem_dir = omm_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, interchange in ic_dict.items():
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        sim_file_dir = mol_dir / f'{mol_name}_sim'
        sim_file_dir.mkdir(exist_ok=True, parents=True)

        # creating simulation and associated files
        sim = ens_fac.create_simulation(interchange, sim_params)
        sim_paths = preparation.prepare_simulation_paths(output_folder=sim_file_dir, output_name=mol_name, sim_params=sim_params)
        reporters = preparation.prepare_simulation_reporters(sim_paths, sim_params)
        preparation.config_simulation(sim, reporters, checkpoint_path=sim_paths.checkpoint)

        # energy min and single-step integration
        sim.minimizeEnergy()
        sim.step(1)

        # extracting energies

        state_data = pd.read_csv(sim_paths.state_data)
        energies = {}
        for key in ('Potential Energy (kJ/mole)', 'Kinetic Energy (kJ/mole)'):
            tag = re.search(STRIP_BEFORE_PARENS, key).group(0)
            E_kj_val = state_data[key][0]
            E_kj = E_kj_val * kilojoule_per_mole
            E_kcal = E_kj.in_units_of(kilocalorie_per_mole)

            for energy in (E_kj, E_kcal):
                energies[f'{tag} ({energy.unit.get_symbol()})'] = energy._value
        
        data_dict.update(**energies)
        data_by_mol.append(data_dict)

# collate energies into DataFrame        
df = pd.DataFrame.from_records(data_by_mol)
df = df.sort_values('Molecule')

# round energy values down to desired precision
round_fn = lambda x : round(x, PRECISION)

for col_name, col in df.items():
    try:
        df[col_name] = col.apply(round_fn) # attempt to round column and replace with rounded values
    except TypeError:
        pass

# save energies to file
energy_file = omm_dir / 'energies_1_step.csv'
df.to_csv(energy_file, index=False)

# Inspecting which molecules fail and succeed and why

In [ ]:
import re
regex = re.compile(r'\A(\w+_\d+)')

diff = {}
for chemistry, succ_names in success_ics.items():
    chem_dir = coll_path / chemistry
    for sdf_path in chem_dir.glob('**/*.sdf'):
        mol_name = re.search(regex, sdf_path.stem).group()
        if mol_name not in succ_names:
            diff.append(mol_name)

## Checking for successful residue covers of newly-generated PDB Topologies

In [ ]:
chemistry = 'urethane'

mgr = PolymerManager(coll_path / chemistry)
mol_names = failed_interchanges[chemistry]['UnmatchedAtomsError']

offmols = {
    mol_name : mgr.polymers[mol_name].offmol_matched(strict=False)
        for mol_name in mol_names
}

In [ ]:
sizes = {
    mol_name : offmol.n_atoms
        for mol_name, offmol in sorted(offmols.items(), key=lambda x : x[1].n_atoms)
}

In [ ]:
sizes

In [ ]:

pdir = mgr.polymers['urethane_6']
# pdir = mgr.polymers['NIPU_8']
# pdir.offmol_matched(strict=True)

for atom in offmol.atoms:
    if not atom.metadata['already_matched']:
        print(atom.metadata)

In [ ]:
mgr = PolymerManager(coll_path / 'NIPU')

offmols = {}
unmatched = []
for mol_name, polymer in mgr.polymers.items():
    try:
        offmols[mol_name] = polymer.offmol
    except:
        unmatched.append(mol_name)

In [ ]:
for mol_name in unmatched:
    print(mol_name)
    polymer = mgr.polymers[mol_name]
    offmol = polymer.offmol_matched(strict=False)
    
    for atom in offmol.atoms:
        if not atom.metadata['already_matched']:
            print('\t', atom.metadata)

# Experimenting with SDF files

In [ ]:
benz = Chem.MolFromSmiles('C1ccccC=1')
benz = Chem.AddHs(benz)
benz.SetDoubleProp('stuff', 3.14)
benz

In [ ]:
block2k = Chem.MolToMolFile(benz, 'test_2k.sdf')
block3k = Chem.MolToV3KMolFile(benz, 'test_3k.sdf')

In [ ]:
block2kforce = Chem.MolToMolFile(benz, 'test_2k_force.sdf', forceV3000=True)

In [ ]:
with Chem.SDWriter('test_sdw.sdf') as sdwriter:
    sdwriter.SetForceV3000(True)
    print(sdwriter.GetForceV3000())

    sdwriter.write(benz)

In [ ]:
with Chem.SDMolSupplier('sdf_testing/test_off_rd.sdf', sanitize=False) as suppl:
    mols = [mol for mol in suppl]

targ = mols[0]
targ

In [ ]:
omol = Molecule.from_rdkit(benz)
omol.generate_conformers(n_conformers=1)
omol.visualize(backend='nglview')

In [ ]:
from polysaccharide import filetree
from polysaccharide import TOOLKITS


p = Path('sdf_testing/test_off_rd.sdf')
tkwrap = TOOLKITS['The RDKit']

omol.properties['series'] = (1,2,3)
omol.to_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load = Molecule.from_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load.properties

In [ ]:
help(tkwrap.to_file)

In [ ]:
help(omol.to_file)